In [2]:
# import ngsolve and webgui
from ngsolve import *
from ngsolve.webgui import Draw

# draw the unit_square shape
Draw(unit_square.shape);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.5, 0.5000000000000001, 0…

In [3]:
# import occ geometry tools
from netgen.occ import *

unit_square_wp = Rectangle(1,1) # this actually returns a WorkPlane object
Draw(unit_square_wp.Face());

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.5, 0.5000000000000001, 0…

In [4]:
geo = OCCGeometry(unit_square.shape, dim = 2) # explicitely state dimension, otherwise dx integrals will vanish
mesh = Mesh(geo.GenerateMesh(maxh = 0.02))
Draw(mesh);

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2401-4-g6fc35016a', 'mesh_dim': 2, 'order2d': …

In [8]:
V = H1(mesh)
gfu = GridFunction(V)

print(gfu.vec)

       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
 

In [9]:
scene = Draw (gfu)

from time import sleep

for i in range(5):
    gfu.vec[:] = 0
    gfu.vec[i] = 1
    scene.Redraw()
    sleep(0.5)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2401-4-g6fc35016a', 'mesh_dim': 2, 'order2d': …

In [10]:
u,u_ = V.TnT() # get test and trial functions

M = BilinearForm(u*u_*dx).Assemble()
S = BilinearForm(grad(u)*grad(u_)*dx).Assemble()

print(M.mat)

Row 0:   0: 3.33333e-05   4: 1.66667e-05   199: 1.66667e-05
Row 1:   1: 5.43825e-05   52: 1.3647e-05   53: 1.35442e-05   248: 2.71912e-05
Row 2:   2: 3.33333e-05   101: 1.66667e-05   102: 1.66667e-05
Row 3:   3: 5.45788e-05   150: 1.37133e-05   151: 1.35761e-05   345: 2.72894e-05
Row 4:   0: 1.66667e-05   4: 0.000126312   5: 2.16936e-05   199: 4.14625e-05   200: 4.64894e-05
Row 5:   4: 2.16936e-05   5: 0.000130013   6: 1.85913e-05   200: 4.6415e-05   201: 4.33127e-05
Row 6:   5: 1.85913e-05   6: 0.000109688   7: 1.71912e-05   201: 3.7653e-05   202: 3.62529e-05
Row 7:   6: 1.71912e-05   7: 0.000101332   8: 1.63335e-05   202: 3.43325e-05   203: 3.34748e-05
Row 8:   7: 1.63335e-05   8: 9.64212e-05   9: 1.57447e-05   203: 3.24659e-05   204: 3.18771e-05
Row 9:   8: 1.57447e-05   9: 9.33038e-05   10: 1.53695e-05   204: 3.12824e-05   205: 3.09072e-05
Row 10:   9: 1.53695e-05   10: 9.12756e-05   11: 1.5096e-05   205: 3.05418e-05   206: 3.02683e-05
Row 11:   10: 1.5096e-05   11: 8.98182e-05   1

In [11]:
u0 = exp(-20*(x**2+y**2))
gfu.Set(u0)
Draw(gfu);

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2401-4-g6fc35016a', 'mesh_dim': 2, 'order2d': …

In [12]:
tau = 0.01
mstarinv = BilinearForm(u*u_*dx+tau**2/4* grad(u)*grad(u_)*dx).Assemble().mat.Inverse()

In [13]:
T = 10
udot = gfu.vec.CreateVector()
udot[:] = 0.
gfu.Set(u0)

scene = Draw(gfu, deformation=True)
for j in range(int(T/tau)):
    gfu.vec.data += tau/2 * udot
    udot.data -= tau * mstarinv*(S.mat*gfu.vec)
    gfu.vec.data += tau/2 * udot
    scene.Redraw()

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2401-4-g6fc35016a', 'mesh_dim': 2, 'order2d': …

## boundary conditions

In [18]:
f = LinearForm(exp(-10*(y-0.5)**2)*u_*ds("left")).Assemble()
omega = 15
T0 = 5
ft = lambda t: cos(t*omega)*(t<T0)

In [19]:
T = 10
udot = gfu.vec.CreateVector()
udot[:] = 0.
gfu.vec[:] = 0.


scene = Draw(gfu, deformation=True)
for j in range(int(T/tau)):
    gfu.vec.data += tau/2 * udot
    udot.data += tau * mstarinv*(1/2*(ft(j*tau)+ft((j+1)*tau))*f.vec-S.mat*gfu.vec)
    gfu.vec.data += tau/2 * udot
    scene.Redraw()

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2401-4-g6fc35016a', 'mesh_dim': 2, 'order2d': …